# Import Dependencies

!pip install matplotlib

In [76]:
import pandas as pd
import json

import matplotlib.pyplot as plt

import numpy as np

# %load_ext nb_black
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql.functions import current_timestamp, count
from pyspark.sql.window import Window

In [77]:
spark = SparkSession.builder \
        .appName("Vinh-Le") \
        .config("fs.s3a.access.key", "${env:AWS_ACCESS_KEY_ID}") \
        .config("fs.s3a.secret.key", "${env:AWS_SECRET_ACCESS_KEY}") \
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
        .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
        .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4') \
        .config('spark.jars.packages', 'org.postgresql:postgresql:42.5.4') \
        .config("fs.s3a.endpoint", "s3.ap-southeast-1.amazonaws.com") \
        .getOrCreate()


spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
spark.conf.set("hive.exec.dynamic.partition", "dynamic")
spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict")
spark.conf.set("spark.sql.broadcastTimeout", 1500)
sc = spark.sparkContext
sc.setSystemProperty("com.amazonaws.services.s3.enableV4", "true")

In [78]:
# Define a function to write the data to PostgreSQL
def write_to_postgres(microBatchDataframe, batchId, tableName):
    # Get the JDBC connection properties
    jdbc_url = "jdbc:postgresql://da-postgres:5432/mydb"
    jdbc_properties = {
        "user": "myuser",
        "password": "mypassword",
        "driver": "org.postgresql.Driver"
    }

    # Write the dataframe to PostgreSQL
    microBatchDataframe.write \
        .mode("append") \
        .jdbc(url=jdbc_url, table=tableName, properties=jdbc_properties)


In [79]:
from pyspark.sql.types import StructType, StructField, StringType, LongType

schema = StructType([
    StructField("account-id", LongType(), True),
    StructField("action", StringType(), True),
    StructField("az-id", StringType(), True),
    StructField("dstaddr", StringType(), True),
    StructField("dstport", LongType(), True),
    StructField("srcaddr", StringType(), True),
    StructField("srcport", LongType(), True),
    StructField("start", LongType(), True),
    StructField("end", LongType(), True),
    StructField("log-status", StringType(), True),
    StructField("packets", LongType(), True),
    StructField("protocol", LongType(), True),
    StructField("region", StringType(), True),
    StructField("subnet-id", StringType(), True),
    StructField("tcp-flags", LongType(), True),
    StructField("traffic-path", StringType(), True),
    StructField("type", StringType(), True),
    StructField("vpc-id", StringType(), True)
])



In [80]:
streaming = (
    spark.readStream.schema(schema)
    .option("maxFilesPerTrigger", 1)
    .csv("s3a://cdp-dev-env-dq/spark-streaming/AWSLogs/621074188511/vpcflowlogs/ap-southeast-1/*/*/*/", schema=schema, header = True, sep = ' ')
)

In [81]:
from pyspark.sql.functions import from_unixtime, col, to_date, to_timestamp, date_format


In [82]:
protocol_dict = {
    '1': 'ICMP',
    '2': 'IGMP', 
    '3': 'GGP',
    '4': 'IP-in-IP',
    '6': 'TCP',
    '8': 'EGP',
    '9': 'IGP',
    '17': 'UDP',
    '41': 'IPv6',
    '47': 'GRE',
    '50': 'ESP',
    '51': 'AH',
    '58': 'ICMPv6',
    '88': 'EIGRP',
    '89': 'OSPF',
    '92': 'MSP',
    '103': 'PIM',
    '112': 'VRRP',
    '115': 'L2TP',
    '132': 'SCTP',
    '136': 'UDPLite'
}

In [83]:
from pyspark.sql.functions import when, col, floor

cleaned_streaming = streaming.withColumn("start", from_unixtime("start").cast("timestamp")) \
                             .withColumn("end", from_unixtime("end").cast("timestamp")) \
                             .withColumn("protocol", when(col("protocol") == "1", "ICMP")
                                         .when(col("protocol") == "2", "IGMP")
                                         .when(col("protocol") == "3", "GGP")
                                         .when(col("protocol") == "4", "IP-in-IP")
                                         .when(col("protocol") == "6", "TCP")
                                         .when(col("protocol") == "8", "EGP")
                                         .when(col("protocol") == "9", "IGP")
                                         .when(col("protocol") == "17", "UDP")
                                         .when(col("protocol") == "41", "IPv6")
                                         .when(col("protocol") == "47", "GRE")
                                         .when(col("protocol") == "50", "ESP")
                                         .when(col("protocol") == "51", "AH")
                                         .when(col("protocol") == "58", "ICMPv6")
                                         .when(col("protocol") == "88", "EIGRP")
                                         .when(col("protocol") == "89", "OSPF")
                                         .when(col("protocol") == "92", "MSP")
                                         .when(col("protocol") == "103", "PIM")
                                         .when(col("protocol") == "112", "VRRP")
                                         .when(col("protocol") == "115", "L2TP")
                                         .when(col("protocol") == "132", "SCTP")
                                         .when(col("protocol") == "136", "UDPLite")
                                         .otherwise("Unknown"))

In [84]:
from pyspark.sql.functions import col, when, split, substring

# Create the boolean conditions
accepted_idx = (col("action") == "ACCEPT")
rejected_idx = (col("action") == "REJECT")

dst_ip = split(col("dstaddr"), "\\.")
src_ip = split(col("srcaddr"), "\\.")
dst_idx = (substring(col("dstaddr"), 1, 4) == "10.0")
src_idx = (substring(col("srcaddr"), 1, 4) == "10.0")

in_idx = ~dst_idx & src_idx
out_idx = ~src_idx & dst_idx
local_idx = src_idx & dst_idx



In [85]:
# Apply the boolean conditions to the DataFrame|
indexed_streaming = cleaned_streaming.withColumn("accepted_idx", accepted_idx) \
                                    .withColumn("rejected_idx", rejected_idx) \
                                    .withColumn("out_idx", out_idx) \
                                    .withColumn("in_idx", in_idx) \
                                    .withColumn("local_idx", local_idx)

In [86]:


# Define the streaming DataFrame with a watermark
local_traffic = indexed_streaming.filter(indexed_streaming.local_idx == True) \
                .withWatermark("start", "1 minute") \
                .groupBy("start") \
                .count()

accepted_in_traffic = indexed_streaming.filter(indexed_streaming.in_idx == True).filter(indexed_streaming.accepted_idx == True) \
                                                .withWatermark("start", "1 minute") \
                                                .groupBy('start').count()

rejected_in_traffic = indexed_streaming.filter(indexed_streaming.in_idx == True).filter(indexed_streaming.rejected_idx == True)\
                                                .withWatermark("start", "1 minute") \
                                                .groupBy('start').count()

accepted_out_traffic = indexed_streaming.filter(indexed_streaming.out_idx == True).filter(indexed_streaming.accepted_idx == True)\
                                                .withWatermark("start", "1 minute") \
                                                .groupBy('start').count()


rejected_out_traffic = indexed_streaming.filter(indexed_streaming.out_idx == True).filter(indexed_streaming.rejected_idx == True)\
                                                .withWatermark("start", "1 minute") \
                                                .groupBy('start').count()

top10_in_traffic = indexed_streaming.filter(indexed_streaming.in_idx == True).filter(indexed_streaming.accepted_idx == True)\
                                                .withWatermark("start", "1 minute") \
                                                .groupBy('srcaddr', 'start').count()
top10_out_traffic = indexed_streaming.filter(indexed_streaming.out_idx == True).filter(indexed_streaming.accepted_idx == True)\
                                                .withWatermark("start", "1 minute") \
                                                .groupBy('srcaddr', 'start').count()

protocol_stream = indexed_streaming.select('protocol', 'start')\
                                    .withWatermark("start", "1 minute") \
                                    .groupBy('protocol', 'start').count()

In [87]:


# Write the stream to PostgreSQL
protocol_query = (
    protocol_stream.writeStream
    .format("foreach")
    .foreachBatch(lambda df, id: write_to_postgres(df, id, "protocol"))
    .option("checkpointLocation", "streaming_s3/protocol/checkpoint/")
    .outputMode("update")
    .start()
)

# protocol_query.awaitTermination()



24/06/07 10:31:51 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [88]:


# Write the stream to PostgreSQL
local_query = (
    local_traffic.writeStream
    .format("foreach")
    .foreachBatch(lambda df, id: write_to_postgres(df, id, "local"))
    .option("checkpointLocation", "streaming_s3/local/checkpoint/")
    .outputMode("update")
    .start()
)


24/06/07 10:31:51 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [89]:


# Write the stream to PostgreSQL
rejected_in_query = (
    rejected_in_traffic.writeStream
    .format("foreach")
    .foreachBatch(lambda df, id: write_to_postgres(df, id, "rejected_in"))
    .option("checkpointLocation", "streaming_s3/rejected_in/checkpoint/")
    .outputMode("update")
    .start()
)

# local_query.awaitTermination()



24/06/07 10:31:52 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [90]:
# Write the stream to PostgreSQL
accepted_in_query = (
    accepted_in_traffic.writeStream
    .format("foreach")
    .foreachBatch(lambda df, id: write_to_postgres(df, id, "accepted_in"))
    .option("checkpointLocation", "streaming_s3/accepted_in/checkpoint/")
    .outputMode("update")
    .start()
)

# local_query.awaitTermination()



24/06/07 10:31:52 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [91]:

# Write the stream to PostgreSQL
accepted_out_query = (
    accepted_out_traffic.writeStream
    .format("foreach")
    .foreachBatch(lambda df, id: write_to_postgres(df, id, "accepted_out"))
    .option("checkpointLocation", "streaming_s3/accepted_out/checkpoint/")
    .outputMode("update")
    .start()
)

# local_query.awaitTermination()



24/06/07 10:31:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [92]:

# Write the stream to PostgreSQL
rejected_out_query = (
    rejected_out_traffic.writeStream
    .format("foreach")
    .foreachBatch(lambda df, id: write_to_postgres(df, id, "rejected_out"))
    .option("checkpointLocation", "streaming_s3/rejected_out/checkpoint/")
    .outputMode("update")
    .start()
)


# local_query.awaitTermination()



24/06/07 10:31:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [93]:
# Write the stream to PostgreSQL
top10_in_query = (
    top10_in_traffic.writeStream
    .format("foreach")
    .foreachBatch(lambda df, id: write_to_postgres(df, id, "top10_in"))
    .option("checkpointLocation", "streaming_s3/top10_in/checkpoint/")
    .outputMode("update")
    .start()
)

# local_query.awaitTermination()



24/06/07 10:31:54 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [94]:
# Write the stream to PostgreSQL
top10_out_query = (
    top10_out_traffic.writeStream
    .format("foreach")
    .foreachBatch(lambda df, id: write_to_postgres(df, id, "top10_out"))
    .option("checkpointLocation", "streaming_s3/top10_out/checkpoint/")
    .outputMode("update")
    .start()
)

# local_query.awaitTermination()



24/06/07 10:31:54 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [95]:
# Transform the streaming DataFrame into a streaming DataFrame with two fields: type_of_packet and count
packet_type_count = indexed_streaming \
    .withColumn("type_of_packet", when(col("in_idx") == "true", "inbound")
                .when(col("out_idx") == "true", "outbound")
                .when(col("local_idx") == "true", "local")
                .otherwise("unknown")) \
    .withWatermark("start", "1 minute") \
    .groupBy("type_of_packet", "start") \
    .agg(count("*").alias("count")) \
    .select("type_of_packet", "start", 'count')

In [96]:

# Write the stream to PostgreSQL
packet_type_query = (
    packet_type_count.writeStream
    .format("foreach")
    .foreachBatch(lambda df, id: write_to_postgres(df, id, "type"))
    .option("checkpointLocation", "streaming_s3/packet_type/checkpoint/")
    .outputMode("update")
    .start()
)
# local_query.awaitTermination()

24/06/07 10:31:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [75]:
top10_out_query.stop()
top10_in_query.stop()
packet_type_query.stop()
rejected_out_query.stop()
accepted_in_query.stop()
accepted_out_query.stop()
rejected_in_query.stop()
local_query.stop()
protocol_query.stop()


24/06/07 07:44:46 WARN TaskSetManager: Lost task 132.0 in stage 151.0 (TID 13502) (172.26.0.5 executor 0): TaskKilled (Stage cancelled: Job 75 cancelled part of cancelled job group a1ff71b8-384f-4525-996f-1a86e4c07287)
24/06/07 07:44:46 WARN TaskSetManager: Lost task 135.0 in stage 151.0 (TID 13505) (172.26.0.5 executor 0): TaskKilled (Stage cancelled: Job 75 cancelled part of cancelled job group a1ff71b8-384f-4525-996f-1a86e4c07287)
24/06/07 07:44:46 WARN TaskSetManager: Lost task 133.0 in stage 151.0 (TID 13503) (172.26.0.5 executor 0): TaskKilled (Stage cancelled: Job 75 cancelled part of cancelled job group a1ff71b8-384f-4525-996f-1a86e4c07287)
24/06/07 07:44:46 WARN TaskSetManager: Lost task 134.0 in stage 151.0 (TID 13504) (172.26.0.5 executor 0): TaskKilled (Stage cancelled: Job 75 cancelled part of cancelled job group a1ff71b8-384f-4525-996f-1a86e4c07287)
24/06/07 10:05:19 ERROR TaskSchedulerImpl: Lost executor 0 on 172.26.0.5: worker lost: Not receiving heartbeat for 60 second

In [101]:
packet_type_query.status['message']

'Terminated with exception: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):\n  File "/usr/local/lib/python3.11/site-packages/py4j/clientserver.py", line 617, in _call_proxy\n    return_value = getattr(self.pool[obj_id], method)(*params)\n                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/spark/python/pyspark/sql/utils.py", line 120, in call\n    raise e\n  File "/opt/spark/python/pyspark/sql/utils.py", line 117, in call\n    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)\n  File "/tmp/ipykernel_3403/723357393.py", line 5, in <lambda>\n    .foreachBatch(lambda df, id: write_to_postgres(df, id, "type"))\n                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/tmp/ipykernel_3403/1940519892.py", line 14, in write_to_postgres\n    .jdbc(url=jdbc_url, table=tableName, properties=jdbc_properties)\n     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n  File "/opt/spark/pyth

24/06/07 10:38:43 WARN TaskSetManager: Lost task 61.0 in stage 201.0 (TID 18059) (172.26.0.5 executor 1): java.sql.BatchUpdateException: Batch entry 0 INSERT INTO rejected_out ("start","count") VALUES ('2024-06-07 10:32:12+00',16) was aborted: ERROR: duplicate key value violates unique constraint "rejected_out_pkey"
  Detail: Key (start)=(2024-06-07 10:32:12) already exists.  Call getNextException to see other errors in the batch.
	at org.postgresql.jdbc.BatchResultHandler.handleError(BatchResultHandler.java:165)
	at org.postgresql.core.ResultHandlerDelegate.handleError(ResultHandlerDelegate.java:52)
	at org.postgresql.core.v3.QueryExecutorImpl.processResults(QueryExecutorImpl.java:2367)
	at org.postgresql.core.v3.QueryExecutorImpl.execute(QueryExecutorImpl.java:560)
	at org.postgresql.jdbc.PgStatement.internalExecuteBatch(PgStatement.java:893)
	at org.postgresql.jdbc.PgStatement.executeBatch(PgStatement.java:916)
	at org.postgresql.jdbc.PgPreparedStatement.executeBatch(PgPreparedState

In [105]:
protocol_query = (
    protocol_stream.writeStream.queryName("protocol_counts")
    .format("console")
    .outputMode("update")
    .start()
)


24/06/07 10:39:53 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6f1295db-7525-47b7-9f15-4fdaf564646a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/07 10:39:53 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------+-------------------+-----+
|protocol|              start|count|
+--------+-------------------+-----+
|     TCP|2024-06-07 10:28:05|    2|
|     TCP|2024-06-07 10:29:19|    6|
|     UDP|2024-06-07 10:28:36|    1|
|     TCP|2024-06-07 10:29:26|   34|
| Unknown|2024-06-07 10:28:57|    1|
|     TCP|2024-06-07 10:27:44|    3|
|     TCP|2024-06-07 10:29:29|    2|
| Unknown|2024-06-07 10:27:52|    1|
| Unknown|2024-06-07 10:28:20|    1|
| Unknown|2024-06-07 10:27:24|    2|
| Unknown|2024-06-07 10:27:09|    1|
| Unknown|2024-06-07 10:28:13|    1|
| Unknown|2024-06-07 10:28:34|    4|
|     TCP|2024-06-07 10:29:33|   30|
| Unknown|2024-06-07 10:27:06|    3|
|     TCP|2024-06-07 10:29:13|    1|
|     TCP|2024-06-07 10:29:27|   29|
|     TCP|2024-06-07 10:28:57|   60|
| Unknown|2024-06-07 10:27:28|    1|
| Unknown|2024-06-07 10:28:52|    1|
+--------+-------------------+-----+
only showing to

-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-------------------+-----+
|protocol|              start|count|
+--------+-------------------+-----+
| Unknown|2024-06-07 10:30:36|    1|
| Unknown|2024-06-07 10:31:56|    1|
|     UDP|2024-06-07 10:32:27|    2|
| Unknown|2024-06-07 10:28:57|    2|
|     TCP|2024-06-07 10:30:39|   12|
| Unknown|2024-06-07 10:31:20|    2|
| Unknown|2024-06-07 10:30:55|    3|
|     TCP|2024-06-07 10:30:19|    8|
|     TCP|2024-06-07 10:32:29|    6|
| Unknown|2024-06-07 10:31:28|    4|
| Unknown|2024-06-07 10:30:16|    2|
|     UDP|2024-06-07 10:30:40|    2|
|     UDP|2024-06-07 10:31:12|    2|
| Unknown|2024-06-07 10:29:44|    3|
|    ICMP|2024-06-07 10:30:04|    1|
|     TCP|2024-06-07 10:32:10|   73|
| Unknown|2024-06-07 10:29:42|    1|
| Unknown|2024-06-07 10:30:29|    1|
| Unknown|2024-06-07 10:32:17|    2|
| Unknown|2024-06-07 10:29:35|    1|
+--------+-------------------+-----+
only showing to

-------------------------------------------
Batch: 2
-------------------------------------------
+--------+-------------------+-----+
|protocol|              start|count|
+--------+-------------------+-----+
|    IPv6|2024-06-07 10:33:43|    1|
| Unknown|2024-06-07 10:33:26|    2|
| Unknown|2024-06-07 10:32:07|    1|
|     TCP|2024-06-07 10:33:46|   27|
|    ICMP|2024-06-07 10:34:28|    1|
| Unknown|2024-06-07 10:32:39|    4|
|     TCP|2024-06-07 10:33:14|    8|
|     TCP|2024-06-07 10:31:56|   21|
| Unknown|2024-06-07 10:33:25|    2|
| Unknown|2024-06-07 10:34:21|    1|
|     TCP|2024-06-07 10:32:10|   80|
|     TCP|2024-06-07 10:33:12|   12|
|     TCP|2024-06-07 10:33:45|   26|
| Unknown|2024-06-07 10:34:05|    2|
| Unknown|2024-06-07 10:34:02|    2|
|     TCP|2024-06-07 10:34:48|    1|
|     UDP|2024-06-07 10:34:41|    1|
|     TCP|2024-06-07 10:32:53|    7|
|     TCP|2024-06-07 10:33:35|   11|
|     TCP|2024-06-07 10:33:29|    9|
+--------+-------------------+-----+
only showing to

-------------------------------------------
Batch: 3
-------------------------------------------
+--------+-------------------+-----+
|protocol|              start|count|
+--------+-------------------+-----+
|     TCP|2024-06-07 10:34:57|   25|
|    ICMP|2024-06-07 10:35:36|    2|
|    IPv6|2024-06-07 10:35:49|    1|
|     TCP|2024-06-07 10:37:20|    2|
| Unknown|2024-06-07 10:35:40|    1|
|     UDP|2024-06-07 10:34:41|    3|
|     TCP|2024-06-07 10:35:13|   18|
|     TCP|2024-06-07 10:35:16|   23|
| Unknown|2024-06-07 10:35:18|    3|
| Unknown|2024-06-07 10:34:38|    1|
|     TCP|2024-06-07 10:35:51|    2|
|     TCP|2024-06-07 10:35:54|   21|
| Unknown|2024-06-07 10:36:17|    3|
| Unknown|2024-06-07 10:36:30|    1|
|     TCP|2024-06-07 10:35:47|   33|
|     TCP|2024-06-07 10:37:37|    1|
|     UDP|2024-06-07 10:37:40|    2|
| Unknown|2024-06-07 10:35:53|    1|
|     TCP|2024-06-07 10:37:42|   29|
| Unknown|2024-06-07 10:34:55|    2|
+--------+-------------------+-----+
only showing to

...

-------------------------------------------
Batch: 11
-------------------------------------------
+--------+-------------------+-----+
|protocol|              start|count|
+--------+-------------------+-----+
| Unknown|2024-06-07 10:47:14|    1|
|     TCP|2024-06-07 10:48:08|    7|
|     TCP|2024-06-07 10:49:03|    1|
| Unknown|2024-06-07 10:48:31|    1|
|    ICMP|2024-06-07 10:49:18|    1|
| Unknown|2024-06-07 10:47:32|    1|
| Unknown|2024-06-07 10:47:34|    1|
|     TCP|2024-06-07 10:48:53|    1|
| Unknown|2024-06-07 10:49:08|    2|
|     TCP|2024-06-07 10:47:53|    2|
| Unknown|2024-06-07 10:48:38|    1|
| Unknown|2024-06-07 10:49:21|    2|
| Unknown|2024-06-07 10:49:18|    3|
|     TCP|2024-06-07 10:49:41|    3|
| Unknown|2024-06-07 10:47:44|    3|
|     TCP|2024-06-07 10:48:04|    1|
| Unknown|2024-06-07 10:49:25|    1|
| Unknown|2024-06-07 10:47:28|    2|
|     TCP|2024-06-07 10:48:36|   24|
|     TCP|2024-06-07 10:47:42|    4|
+--------+-------------------+-----+
only showing t

-------------------------------------------
Batch: 12
-------------------------------------------
+--------+-------------------+-----+
|protocol|              start|count|
+--------+-------------------+-----+
| Unknown|2024-06-07 10:50:00|    1|
|     TCP|2024-06-07 10:52:16|   28|
|     TCP|2024-06-07 10:52:42|   63|
| Unknown|2024-06-07 10:50:02|    1|
|     TCP|2024-06-07 10:50:56|    3|
| Unknown|2024-06-07 10:51:10|    3|
| Unknown|2024-06-07 10:49:34|    2|
|     TCP|2024-06-07 10:51:31|    8|
| Unknown|2024-06-07 10:51:38|    1|
| Unknown|2024-06-07 10:50:07|    1|
|     TCP|2024-06-07 10:50:32|    2|
|     TCP|2024-06-07 10:52:02|    4|
| Unknown|2024-06-07 10:50:13|    3|
| Unknown|2024-06-07 10:51:52|    1|
|     TCP|2024-06-07 10:52:48|   13|
| Unknown|2024-06-07 10:50:56|    2|
| Unknown|2024-06-07 10:52:13|    1|
| Unknown|2024-06-07 10:50:09|    5|
| Unknown|2024-06-07 10:49:08|    3|
|     TCP|2024-06-07 10:51:25|    2|
+--------+-------------------+-----+
only showing t

-------------------------------------------
Batch: 13
-------------------------------------------
+--------+-----+-----+
|protocol|start|count|
+--------+-----+-----+
+--------+-----+-----+



-------------------------------------------
Batch: 14
-------------------------------------------
+--------+-------------------+-----+
|protocol|              start|count|
+--------+-------------------+-----+
| Unknown|2024-06-07 10:53:43|    1|
| Unknown|2024-06-07 10:54:53|    1|
| Unknown|2024-06-07 10:54:44|    3|
| Unknown|2024-06-07 10:56:05|    1|
|     UDP|2024-06-07 10:57:20|    3|
|     TCP|2024-06-07 10:55:02|   11|
| Unknown|2024-06-07 10:54:14|    1|
|     TCP|2024-06-07 10:56:14|    6|
|     TCP|2024-06-07 10:56:32|    9|
| Unknown|2024-06-07 10:57:00|    1|
|     UDP|2024-06-07 10:55:17|    2|
| Unknown|2024-06-07 10:54:08|    3|
| Unknown|2024-06-07 10:55:01|    2|
|     TCP|2024-06-07 10:55:20|    1|
|    ICMP|2024-06-07 10:56:32|    1|
|     TCP|2024-06-07 10:55:27|   34|
| Unknown|2024-06-07 10:53:50|    1|
|     TCP|2024-06-07 10:55:28|   33|
| Unknown|2024-06-07 10:55:43|    4|
|     TCP|2024-06-07 10:55:07|    8|
+--------+-------------------+-----+
only showing t

-------------------------------------------
Batch: 15
-------------------------------------------
+--------+-------------------+-----+
|protocol|              start|count|
+--------+-------------------+-----+
|     TCP|2024-06-07 10:52:42|   65|
| Unknown|2024-06-07 10:53:43|    4|
|     TCP|2024-06-07 10:53:53|   13|
|    ICMP|2024-06-07 10:53:46|    1|
| Unknown|2024-06-07 10:53:32|    3|
| Unknown|2024-06-07 10:53:31|    1|
|     TCP|2024-06-07 10:53:10|   74|
| Unknown|2024-06-07 10:52:56|    1|
|     UDP|2024-06-07 10:53:07|    1|
| Unknown|2024-06-07 10:54:17|    1|
|     TCP|2024-06-07 10:54:32|    4|
|    ICMP|2024-06-07 10:53:38|    2|
| Unknown|2024-06-07 10:54:08|    5|
|     TCP|2024-06-07 10:54:17|    5|
|     TCP|2024-06-07 10:52:48|   34|
| Unknown|2024-06-07 10:53:50|    2|
| Unknown|2024-06-07 10:52:35|    1|
|     TCP|2024-06-07 10:53:44|   22|
| Unknown|2024-06-07 10:54:16|    1|
|     UDP|2024-06-07 10:53:32|    1|
+--------+-------------------+-----+
only showing t